In [3]:
from typing import List
from sqlalchemy import create_engine
from sqlalchemy import ForeignKey
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship
from sqlalchemy.orm import Session
class Base(DeclarativeBase):
    pass
class Company(Base):
    __tablename__ = "company"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str]
    employees: Mapped[List["Employee"]] = relationship(back_populates="company")
class Employee(Base):
    __tablename__ = "employee"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str]
    type: Mapped[str]
    company_id: Mapped[int] = mapped_column(ForeignKey("company.id"))
    company: Mapped[Company] = relationship(back_populates="employees")
    def __repr__(self):
        return f"{self.__class__.__name__}({self.name!r})"
    __mapper_args__ = {
        "polymorphic_identity": "employee",
        "polymorphic_on": "type",
    }
class Manager(Employee):
    __tablename__ = "manager"
    id: Mapped[int] = mapped_column(ForeignKey("employee.id"), primary_key=True)
    manager_name: Mapped[str]
    paperwork: Mapped[List["Paperwork"]] = relationship()
    __mapper_args__ = {
        "polymorphic_identity": "manager",
    }
class Paperwork(Base):
    __tablename__ = "paperwork"
    id: Mapped[int] = mapped_column(primary_key=True)
    manager_id: Mapped[int] = mapped_column(ForeignKey("manager.id"))
    document_name: Mapped[str]
    def __repr__(self):
        return f"Paperwork({self.document_name!r})"
class Engineer(Employee):
    __tablename__ = "engineer"
    id: Mapped[int] = mapped_column(ForeignKey("employee.id"), primary_key=True)
    engineer_info: Mapped[str]
    __mapper_args__ = {
        "polymorphic_identity": "engineer",
    }
engine = create_engine("sqlite:///apps/farmwise.db", echo=True)
Base.metadata.create_all(engine)

conn = engine.connect()
from sqlalchemy.orm import Session
session = Session(conn)
session.add(
    Company(
        name="Krusty Krab",
        employees=[
            Manager(
                name="Mr. Krabs",
                manager_name="Eugene H. Krabs",
                paperwork=[
                    Paperwork(document_name="Secret Recipes"),
                    Paperwork(document_name="Krabby Patty Orders"),
                ],
            ),
            Engineer(name="SpongeBob", engineer_info="Krabby Patty Master"),
            Engineer(
                name="Squidward",
                engineer_info="Senior Customer Engagement Engineer",
            ),
        ],
    )
)
session.commit()

2025-05-08 09:51:19,810 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-08 09:51:19,816 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("company")
2025-05-08 09:51:19,823 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-08 09:51:19,827 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("company")
2025-05-08 09:51:19,830 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-08 09:51:19,831 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("employee")
2025-05-08 09:51:19,832 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-08 09:51:19,833 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("employee")
2025-05-08 09:51:19,833 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-08 09:51:19,834 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("manager")
2025-05-08 09:51:19,834 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-08 09:51:19,835 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("manager")
2025-05-08 09:51:19,836 INFO sqlalchemy.engine.Engine [raw sql] 

In [4]:
from sqlalchemy import select
stmt = select(Manager).order_by(Manager.id)
managers = session.scalars(stmt).all()
print(managers)

2025-05-08 09:51:59,296 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-08 09:51:59,300 INFO sqlalchemy.engine.Engine SELECT manager.id, employee.id AS id_1, employee.name, employee.type, employee.company_id, manager.manager_name 
FROM employee JOIN manager ON employee.id = manager.id ORDER BY manager.id
2025-05-08 09:51:59,301 INFO sqlalchemy.engine.Engine [generated in 0.00068s] ()
[Manager('Mr. Krabs')]


In [5]:
from sqlalchemy import select
stmt = select(Employee).order_by(Employee.id)
objects = session.scalars(stmt).all()
print(objects)

2025-05-08 09:52:32,048 INFO sqlalchemy.engine.Engine SELECT employee.id, employee.name, employee.type, employee.company_id 
FROM employee ORDER BY employee.id
2025-05-08 09:52:32,049 INFO sqlalchemy.engine.Engine [generated in 0.00154s] ()
[Manager('Mr. Krabs'), Engineer('SpongeBob'), Engineer('Squidward')]
